# RRTMGP_RFMIP_LW

*Last edited: 2024-10-30*

This code is part of RRTM for GCM Applications - Parallel (RRTMGP). Please see `rrtmgp_rfmip_lw.F90` for more info.

- Example program to demonstrate the calculation of longwave radiative fluxes in clear, aerosol-free skies.
- The example files come from the Radiative Forcing MIP (https://www.earthsystemcog.org/projects/rfmip/).
- The large problem (1800 profiles) is divided into blocks.
- Program is invoked as rrtmgp_rfmip_lw [block_size input_file coefficient_file upflux_file downflux_file] .
- All arguments are optional but need to be specified in order.

The goal of this notebook is to study the operation of the example program `rrtmgp_rfmip_lw.F90`, investigating and documenting it. Fortran is used through the internal magic commands "%%writefile" and "!" (shell). To create the source code file, all cells containing "%%writefile" must be executed. The code has been divided into small cells, to try to make the parts easier to understand. This is probably not the most elegant way to use Fortran, but for my purposes it works relatively well, and it is possible to perform quick experiments using Python's interactive features. To make it easier to understand, this version does not use OpenACC, OpenMP, Timing, Papi and other optional libraries.

## Table of Contents

> (automatically generated by JupyterLab, use the menu or Ctrl+Shift+K)

## Python libraries

In [12]:
import netCDF4 as nc

## Makefile

The Makefile needs to be edited using an external editor which treats spaces and tabs as meaning different things.

## rrtmgp_rfmip_lw.F90

Note: the original version of `rrtmgp_rfmip_lw.F90` has been renamed to `rrtmgp_rfmip_lw-v1.F90`.

Start of the code

- Error checking: Procedures in rte+rrtmgp return strings which are empty if no errors occured.
- Check the incoming string, print it out and stop execution if non-empty.

Workdir:

In [10]:
%cd /home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky/

/home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky


Filename:

In [ ]:
FILE = "rrtmgp_rfmip_lw.F90"

In [29]:
#23456789 123456789 123456789 123456789 123456789 123456789 123456789 12

In [33]:
%%writefile {FILE}
!# ** rrtmgp_rfmip_lw.F90 **
!# This file was generated by the Notebook `rrtmgp_rfmip_lw.ipynb`, and 
!# it is rewritten each time the Notebook is run, losing all changes 
!# made. To make changes to the file, you must make them in the 
!# Notebook, and rerun it to recreate this file.

Overwriting rrtmgp_rfmip_lw.F90


### Subroutines

In [34]:
%%writefile -a {FILE}

subroutine stop_on_err(error_msg)
  use iso_fortran_env, only : error_unit
  use iso_c_binding
  character(len=*), intent(in) :: error_msg
  if(error_msg /= "") then
    write (error_unit,*) trim(error_msg)
    write (error_unit,*) "Stopping."
    stop
  end if
end subroutine stop_on_err

Appending to rrtmgp_rfmip_lw.F90


### Main program

In [35]:
%%writefile -a {FILE}

program rrtmgp_rfmip_lw

Appending to rrtmgp_rfmip_lw.F90


### Modules for working with rte and rrtmgp

 Working precision for real variables:

In [36]:
%%writefile -a {FILE}

use mo_rte_kind, only: wp, sp, wl, i4

Appending to rrtmgp_rfmip_lw.F90


- Optical properties of the atmosphere as array of values
  - In the longwave we include only absorption optical depth (_1scl)
  - Shortwave calculations would use optical depth, single-scattering albedo, asymmetry parameter (_2str)

In [37]:
%%writefile -a {FILE}

use mo_optical_props, only: ty_optical_props_1scl

Appending to rrtmgp_rfmip_lw.F90


- Gas optics: maps physical state of the atmosphere to optical properties

In [38]:
%%writefile -a {FILE}

use mo_gas_optics_rrtmgp, only: ty_gas_optics_rrtmgp

Appending to rrtmgp_rfmip_lw.F90


- Gas optics uses a derived type to represent gas concentrations compactly...

In [39]:
%%writefile -a {FILE}

 use mo_gas_concentrations, only: ty_gas_concs

Appending to rrtmgp_rfmip_lw.F90


- ... and another type to encapsulate the longwave source functions.

In [40]:
%%writefile -a {FILE}

use mo_source_functions, only: ty_source_func_lw

Appending to rrtmgp_rfmip_lw.F90


- RTE longwave driver.

In [41]:
%%writefile -a {FILE}

use mo_rte_lw, only: rte_lw

Appending to rrtmgp_rfmip_lw.F90


- RTE driver uses a derived type to reduce spectral fluxes to whatever the user wants
  - Here we're just reporting broadband fluxes

In [42]:
%%writefile -a {FILE}

use mo_fluxes, only: ty_fluxes_broadband, ty_fluxes_flexible

Appending to rrtmgp_rfmip_lw.F90


  - modules for reading and writing files
  - RRTMGP's gas optics class needs to be initialized with data read from a netCDF files

In [43]:
%%writefile -a {FILE}

use mo_load_coefficients, only: load_and_init
use mo_rfmip_io, only: read_size, read_and_block_pt, &
  read_and_block_gases_ty, unblock_and_write, unblock, &
  read_and_block_lw_bc, determine_gas_names
use mo_simple_netcdf, only: read_field, get_dim_size
use netcdf
use mod_network_rrtmgp

Appending to rrtmgp_rfmip_lw.F90


The "implicit" declaration has to come after the "use" ones.

In [44]:
%%writefile -a {FILE}

implicit none

Appending to rrtmgp_rfmip_lw.F90


### NN models

- Neural networks for gas optics (optional).
- netCDF files which describe the models and pre-processing coefficients (the two models predict SW absorption and Rayleigh scattering, respectively).
- see more comments in section [Arguments](#Arguments).

In [ ]:
%%writefile -a {FILE}

character(len=80) :: &
  modelfile_tau = &
    "../../neural/data/BEST_tau-lw-18-58-58.nc", &
  modelfile_source = &
    "../../neural/data/BEST_pfrac-18-16-16.nc"

In [9]:
! ls -shw1 ../../neural/data/ \
    > /home/x/git/radnn/aux/neural-data-dir-list.txt

### Local variables

In [45]:
%%writefile -a {FILE}

character(len=132) :: &
  rfmip_file = &
    'multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc', &
  kdist_file = &
    'coefficients_lw.nc'

Appending to rrtmgp_rfmip_lw.F90


In [47]:
%%writefile -a {FILE}

character(len=132) :: flx_file, flx_file_ref, flx_file_lbl, timing_file
integer            :: nargs, ncol, nlay, nbnd, ngpt, nexp, nblocks, &
                      block_size, forcing_index, physics_index, &
                      n_quad_angles = 1
logical            :: top_at_1
integer            :: b, icol, ilay, ibnd, igpt, count_rate, iTime1, &
                      iTime2, iTime3, ncid, ninputs, istat, igas, ret, i
character(len=5)   :: block_size_char, forcing_index_char = '1', &
                      physics_index_char = '1'
character(len=32), dimension(:), allocatable :: kdist_gas_names, &
                      rfmip_gas_names
!# block_size, nlay, nblocks
real(wp), dimension(:,:,:), allocatable :: p_lay, p_lev, t_lay, t_lev
real(wp), dimension(:,:,:), target, allocatable :: flux_up, flux_dn
real(wp), dimension(:,:,:,:), target, allocatable :: gpt_flux_up, &
                      gpt_flux_dn
real(wp), dimension(:,:,:), allocatable :: rlu_ref, rld_ref, rlu_nn, &
                      rld_nn, rlu_lbl, rld_lbl, rldu_ref, rldu_nn, &
                      rldu_lbl, col_dry
!# block_size, nblocks (emissivity is spectrally constant)
real(wp), dimension(:,:), allocatable :: sfc_emis, sfc_t
!# nbands, block_size (spectrally-resolved emissivity)
real(wp), dimension(:,:), allocatable :: sfc_emis_spec
real(wp) :: bb_flux_up
logical :: use_rrtmgp_nn, do_gpt_flux, compare_flux, save_flux

Appending to rrtmgp_rfmip_lw.F90


"neural_nets" can be used with one or two cdf files, depending on the amount of command line arguments when calling "rrtmgp_rfmip_lw" (see "Arguments" below). Classes used by rte+rrtmgp:

In [ ]:
%%writefile -a {FILE}

type(ty_gas_optics_rrtmgp)  :: k_dist
type(ty_source_func_lw)     :: source
type(ty_optical_props_1scl) :: optical_props
type(ty_fluxes_flexible)    :: fluxes

Type used to create NNs. 1 NN is created when the number of command line arguments is 6, and 2 when it is 7.

In [ ]:
%%writefile -a {FILE}

!# First model is for "absorption", second is for "Planck fraction"
type(rrtmgp_network_type), dimension(:), allocatable :: neural_nets

Appending to rrtmgp_rfmip_lw.F90


- `ty_gas_concentration` holds multiple columns; we make an array of these objects to leverage what we know about the input file

In [49]:
%%writefile -a {FILE}

type(ty_gas_concs), dimension(:), allocatable  :: gas_conc_array

Appending to rrtmgp_rfmip_lw.F90


### Code starts

All arguments are optional

### I/O and settings

Use neural networks for gas optics?  if NN models provided, set to true, but can also be overriden:

In [50]:
%%writefile -a {FILE}

use_rrtmgp_nn = .false.

Appending to rrtmgp_rfmip_lw.F90


Save fluxes:

In [51]:
%%writefile -a {FILE}

save_flux = .false.

Appending to rrtmgp_rfmip_lw.F90


Compare fluxes to reference code as well as line-by-line (RFMIP only):

In [52]:
%%writefile -a {FILE}

compare_flux = .false.

Appending to rrtmgp_rfmip_lw.F90


Compute fluxes per g-point? :

In [53]:
%%writefile -a {FILE}

do_gpt_flux = .false.

Appending to rrtmgp_rfmip_lw.F90


### Arguments

In [ ]:
%%writefile -a {FILE}

print *, "** rrtmgp_rfmip_lw.F90 version 2024-10-28 **"
print *, "Usage:"
print *, "rrtmgp_rfmip_lw \"             !# 1
print *, "    [block_size] \"            !# 2
print *, "    [rfmip_file] \"            !# 3
print *, "    [k-distribution_file] \"   !# 4
print *, "    [forcing_index (1,2,3)] \" !# 5
print *, "    [physics_index (1,2)]  \"  !# 6
print *, "    [input_output file]"       !# 7
print *, "OR:"
print *, "rrtmgp_rfmip_lw \"
print *, "    [block_size] \"           !# 1
print *, "    [rfmip_file] \"           !# 2
print *, "    [k-distribution_file] \"  !# 3
print *, "    [forcing_index] \"        !# 4
print *, "    [physics_index] \"        !# 5
print *, "    [NN_lw_abs_file] \"       !# 6
print *, "    [NN_lw_planck_file]"      !# 7

In [ ]:
%%writefile -a {FILE}

nargs = command_argument_count()
call get_command_argument(1, block_size_char)
read(block_size_char, '(i5)') block_size
if(nargs >= 2) call get_command_argument(2, rfmip_file)
if(nargs >= 3) call get_command_argument(3, kdist_file)
if(nargs >= 4) call get_command_argument(4, forcing_index_char)
if(nargs >= 5) call get_command_argument(5, physics_index_char)
if(nargs >= 6) then
  use_rrtmgp_nn = .true.
  !# e.g.: LW absorption model
  call get_command_argument(6, modelfile_tau)
  if (nargs >= 7) then
    allocate(neural_nets(2))
    !# e.g.: Planck fraction model
    call get_command_argument(7, modelfile_source)
  else 
    allocate(neural_nets(1))
  end if
end if

When the number of arguments is greater than 6, the NN version is activated. If 6 arguments are used, then a NN is created for argument 6. If argument 7 is also used, then one more NN is created, totaling 2.

- modelfile_tau = [NN_lw_abs_file]
  - LW absorption model
- modelfile_source = [NN_lw_planck_file]
  - Planck fraction model

NetCDF files which describe the models and pre-processing coefficients. The two models predict, e.g.:

- SW absorption
- Rayleigh scattering

Separate models are trained to predict:

- LW
  - absorption optical depth
  - emission Planck fraction
- SW
  - absorption
  - Rayleigh optical depths

These are multi-output networks which predict all g-points simultaneously, so vectors of sizes 256 (LW) or 224 (SW).

- Number of g-points in the RRTMGP k-distribution
    - 256 within 16 LW bands
    - 224 within 14 SW bands

- RRTMG, the predecessor to RRMTGP widely used in large-scale models, has 140 [LW] and 112 [SW] g-points

---

From [UKK23]:

- "Recently, reduced k distributions with half the number of g points (112/128) [LW/SW] have been generated from the full distributions. This was done using the same approach as in the evolution from the RRTM scheme (Mlawer et al., 1997) to its reduced-resolution version designed for GCMs, RRTMG (Iacono et al., 2000), namely by iteratively combining neighbouring g points while attempting to minimize a cost function that includes fluxes, forcing and heating rates (Pincus & Mlawer, 2022). The reduction in g points was similar in both cases; RRTMG, which is used operationally in the IFS, has 112/140 g points. As the number of g points gives the number of pseudomonochromatic radiative transfer calculations, it largely determines the cost of the whole radiation scheme. The NNs developed in this paper [UKK23] are therefore based on the new reduced k distributions ("**reduced RRTMGP**") and not the k distributions with higher spectral resolution ("full RRTMGP")."

Each of these 2 CDF files are read and placed into their own NN. If both are loaded, 2 NNs are created (see the [Load Neural Network models](#Load-Neural-Network-models) section below).

The variables are predefined above in section [NN models](#NN-models) (these 2 datasets are not publicly available):

- modelfile_tau = "../../neural/data/BEST_tau-lw-18-58-58.nc".
- modelfile_source = "../../neural/data/BEST_pfrac-18-16-16.nc".

Notes for /neural/data/ :

- `tau` = absorption
- The `neural/data/` directory (see [neural-data-dir-list.txt](aux/neural-data-dir-list.txt)) is the same one used by `ukk23test01-train-v2.ipynb`, so it must have something to do with it
- The directory is a mess, there are `lw*absorption*` files that seem to be related to `BEST_tau-lw*`, and `lw*planck_frac*` files that seem to be related to `BEST_pfrac*`.
- Files with `*g112*` and `*g128*` are probably related to "reduced RRTMGP (112/128) [LW/SW]"

The number "18" in the filename "BEST_*-18-*.nc" is likely related to 18 input, see Table 1 below. Let's take these example files:

    100K lw-g256-2018-12-04_absorption_58_58.nc
    4,0K lw-g256-2018-12-04_absorption_BEST.nc
     40K lw-g256-2018-12-04_planck_frac_16_16.nc
    4,0K lw-g256-2018-12-04_planck_frac_BEST.nc
    
     40K sw-g224-2018-12-04-absorption_16_16.nc
    4,0K sw-g224-2018-12-04-absorption_BEST.nc
     40K sw-g224-2018-12-04-rayleigh_16_16.nc
    4,0K sw-g224-2018-12-04-rayleigh_BEST.nc

- `*_BEST` files, despite the extension `.nc`, are text files containing a file name, pointing to one of the other files in the directory:

    - lw-g256-2018-12-04_absorption_BEST.nc = lw-g256-2018-12-04_absorption_58_58.nc
    - lw-g256-2018-12-04_planck_frac_BEST.nc = lw-g256-2018-12-04_planck_frac_16_16.nc
    - sw-g224-2018-12-04-absorption_BEST.nc = sw-g224-2018-12-04-absorption_16_16.nc
    - sw-g224-2018-12-04-rayleigh_BEST.nc = sw-g224-2018-12-04-rayleigh_16_16.nc

Do not confuse *best* with *both*, there are other files with *both* in the name that are used in a unified model.


---

From [UKK20] :

- `BEST_tau-lw-18-58-58.nc` and `BEST_pfrac-18-16-16.nc` seem to be linked to Table 1:

    | Process       | Predicted variable       | Inputs | Neurons | Outputs |
    | ------------- | ------------------------ | ------ | ------- | ------- |
    | LW absorption | absorption cross section | 18     | 58–58   | 256     |
    | LW emission   | Planck fraction          | 18     | 16–16   | 256     |
    | SW absorption | absorption cross section | 7      | 48–48   | 224     |
    | SW scattering | Rayleigh cross section   | 7      | 16–16   | 224     |

- `tau` appears to be related to: "RRTMGP computes absorption optical depth **τ** (tau) as the product of the absorption cross section k (m2 mol<sup>−1</sup>) and the path number of molecules in a column N (mol m<sup>−2</sup>)".

- UKK20 methodology uses two different NNs to forecast the absorption cross section and Planck fractions.

- To choose what variables to predict, we follow the underlying kernels in RRTMGP to respect a physical separation of concerns. Separate models are trained to predict absorption optical depth and Planck fraction in the LW and absorption and Rayleigh optical depths in the SW. These are multi-output networks which predict all g-points simultaneously, so vectors of sizes 256 (LW) or 224 (SW).

- Our accelerated radiation code for dynamical models, RTE+RRTMGP-NN, is like its parent code written in modern Fortran and uses object-oriented programming to provide flexibility and to separate computations from flow control. The NN models are loaded from data files specified at runtime, similar to the k-distribution in RRTMGP.

---

From `examples/rrtmgp-nn-training/`:

- To perform the prediction, "g-point" vectors are used, containing:
    - LW: Planck fraction, absorption cross-section, or both
    - SW: Absorption cross-section, or Rayleigh cross-section
- Configure predictand
    - predictand = "lw_both"

---

From `ml_load_save_preproc.py`:

Apparently this comment explains the `*both*` files:

```python
elif (predictand == 'lw_both'): 
    # I tested just having a unified LW model, didn't seem very promising
```

This code snippet shows the different types of dataset files:

- **load_rrtmgp** : Load data for training a GAS OPTICS (RRTMGP) emulator, where inputs are layer-wise atmospheric conditions (T,p, gas concentrations) and outputs are vectors of optical properties across g-points (e.g. optical depth)

```python
load_rrtmgp(fname,predictand, dcol=1, skip_lastlev=False, skip_firstlev=False,expfirst=False)       
    if predictand not in ['lw_absorption', 'lw_planck_frac', 'sw_absorption', 
                          'sw_rayleigh', 'lw_both']: 
        sys.exit("Second drgument to load_rrtmgp (predictand) " \
        "must be either lw_absorption, lw_planck_frac, sw_absorption, or sw_rayleigh")
```

Apparently "lw_absorption", "lw_planck_frac", "lw_both" are related:

```python
if predictand in ["lw_absorption", "lw_planck_frac",'lw_both']: # Longwave
        xname = 'rrtmgp_lw_input'
    else: # Shortwave
        xname = 'rrtmgp_sw_input'
```

Apparently "lw_both" contains both "lw_absorption" (tau=absorption) and "lw_planck_frac":

```python
elif (predictand=='lw_both'):
    y = dat.variables['tau_lw_gas'][:].data
    y2 = dat.variables['planck_fraction'][:].data   
    y = np.concatenate((y,y2),axis=-1)
```

Dataset file structure for `*absorption*`, `*both*`, and `*plank*`. Apparently `*absorption*` and `*both*` have the same structure:

In [18]:
PATH = "/home/x/git/radnn/ukk23test01/neural/data/"
FILE = "lw-g256-2018-12-04_absorption_58_58.nc"
print(list(nc.Dataset(PATH+FILE).variables))

['nn_dimsize', 'nn_activation', 'nn_inputs', 'nn_input_coeffs_max', 'nn_input_coeffs_min', 'nn_activation_char', 'nn_inputs_char', 'nn_weights_1', 'nn_bias_1', 'nn_weights_2', 'nn_bias_2', 'nn_weights_3', 'nn_bias_3', 'nn_output_coeffs_mean', 'nn_output_coeffs_std']


In [20]:
FILE = "lw-g128-210809_both_56_56_HR_1.11e+00_FRC_7.57e-01.nc"
print(list(nc.Dataset(PATH+FILE).variables))

['nn_dimsize', 'nn_activation', 'nn_inputs', 'nn_input_coeffs_max', 'nn_input_coeffs_min', 'nn_activation_char', 'nn_inputs_char', 'nn_weights_1', 'nn_bias_1', 'nn_weights_2', 'nn_bias_2', 'nn_weights_3', 'nn_bias_3', 'nn_output_coeffs_mean', 'nn_output_coeffs_std']


In [19]:
FILE = "lw-g256-2018-12-04_planck_frac_16_16.nc"
print(list(nc.Dataset(PATH+FILE).variables))

['nn_dimsize', 'nn_activation', 'nn_inputs', 'nn_input_coeffs_max', 'nn_input_coeffs_min', 'nn_activation_char', 'nn_inputs_char', 'nn_weights_1', 'nn_bias_1', 'nn_weights_2', 'nn_bias_2', 'nn_weights_3', 'nn_bias_3']


Continuing the Fortran code:

In [57]:
%%writefile -a {FILE}

call read_size(rfmip_file, ncol, nlay, nexp)

Appending to rrtmgp_rfmip_lw.F90


In [58]:
%%writefile -a {FILE}

if (nexp==18) compare_flux=.true.

print *, "Input file: ", rfmip_file
print *, "nexp:", nexp, "ncol:", ncol, "nlay:", nlay, &
  "block_size:", block_size

if(mod(ncol*nexp, block_size) /= 0 ) &
  call stop_on_err("rrtmgp_rfmip_lw: number of columns doesn't " // &
    "fit evenly into blocks.")

nblocks = (ncol*nexp)/block_size
print *, "Doing ",  nblocks, "blocks of size ", block_size

read(forcing_index_char, '(i4)') forcing_index
if(forcing_index < 1 .or. forcing_index > 4) &
  stop "Forcing index is invalid (must be 1, 2 ou 3)"

read(physics_index_char, '(i4)') physics_index
if(physics_index < 1 .or. physics_index > 2) &
  stop "Physics index is invalid (must be 1 ou 2)"

if(physics_index == 2) n_quad_angles = 3

Appending to rrtmgp_rfmip_lw.F90


Save upwelling and downwelling fluxes in the same file :

In [59]:
%%writefile -a {FILE}

if (use_rrtmgp_nn) then
  flx_file = 'output_fluxes/rlud_Efx_RTE-RRTMGP-NN-181204_rad-irf_' // &
             'r1i1p1f' // trim(forcing_index_char) // '_gn.nc'
else
  flx_file = 'output_fluxes/rlud_Efx_RTE-RRTMGP-181204_rad-irf_' // &
             'r1i1p1f' // trim(forcing_index_char) // '_gn.nc'
end if

Appending to rrtmgp_rfmip_lw.F90


Identify the set of gases used in the calculation based on the forcing index. A gas might have a different name in the k-distribution than in the files provided by RFMIP (e.g. 'co2' and 'carbon_dioxide') :

In [60]:
%%writefile -a {FILE}

call determine_gas_names(rfmip_file, &
                         kdist_file, &
                         forcing_index, &
                         kdist_gas_names, &
                         rfmip_gas_names)

print *, "Calculation uses RFMIP gases: ", &
  (trim(kdist_gas_names(b)) // " ", b = 1, size(kdist_gas_names))

Appending to rrtmgp_rfmip_lw.F90


### Load Neural Network models

- "%" in Fortran allows to create OOP structures. Not to be confused with Jupyter's "%".

In [61]:
%%writefile -a {FILE}

if (use_rrtmgp_nn) then

  print *, 'Loading longwave absorption model from: ', modelfile_tau
  call neural_nets(1) % load_netcdf(modelfile_tau)

  if (nargs >= 7) then 
    print *, 'Loading Planck fraction model from: ', modelfile_source
    call neural_nets(2) % load_netcdf(modelfile_source)
  end if

  ninputs = size(neural_nets(1) % layers(1) % w_transposed, 2)
  print *, "NN supports gases: ", &
    (trim(neural_nets(1) % input_names(b)) // " ", b = 3, &
    size(neural_nets(1) % input_names))

end if

Appending to rrtmgp_rfmip_lw.F90


### Prepare data for use in rte+rrtmgp

Allocation on assignment within reading routines

`read_and_block_pt`

Return layer and level pressures and temperatures as arrays dimensioned (ncol, nlay/+1, nblocks)

- Input arrays are dimensioned (nlay/+1, ncol, nexp)

- Output arrays are allocated within this routine

- intent(out) :: p_lay, p_lev, t_lay, t_lev

- The number of columns should fit evenly into the blocks.

In [62]:
%%writefile -a {FILE}

call read_and_block_pt(rfmip_file, &
                       block_size, &
                       p_lay,      & !# intent(out)
                       p_lev,      & !# intent(out)
                       t_lay,      & !# intent(out)
                       t_lev)        !# intent(out)

!# print *, "shape t_lay, min, max", shape(t_lay), maxval(t_lay), &
!#          minval(t_lay)

Appending to rrtmgp_rfmip_lw.F90


Are the arrays ordered in the vertical with 1 at the top or the bottom of the domain?

In [63]:
%%writefile -a {FILE}

top_at_1 = p_lay(1, 1, 1) < p_lay(nlay, 1, 1)

Appending to rrtmgp_rfmip_lw.F90


Read the gas concentrations and surface properties

In [64]:
%%writefile -a {FILE}

call read_and_block_gases_ty(rfmip_file,      &
                             block_size,      &
                             kdist_gas_names, & 
                             rfmip_gas_names, &
                             gas_conc_array )

!# do b = 1, size(gas_conc_array(1)%concs)
!#  print *, "max of gas ", gas_conc_array(1)%gas_name(b), ":", &
!#    maxval(gas_conc_array(1)%concs(b)%conc)
!# end do

call read_and_block_lw_bc(rfmip_file, &
                          block_size, &
                          sfc_emis,   &
                          sfc_t)

Appending to rrtmgp_rfmip_lw.F90


Read k-distribution information. `load_and_init()` reads data from netCDF and calls `k_dist%init()` ; users might want to use their own reading methods

In [65]:
%%writefile -a {FILE}

call load_and_init(k_dist,           &
                   trim(kdist_file), &
                   gas_conc_array(1))

!# print *, "min of play", minval(p_lay), &
!#  "p_lay = k_dist%get_press_min()", k_dist%get_press_min()
!# print *," press min max", k_dist%get_press_min(), k_dist%get_press_max()
!# print *," temp min max", k_dist%get_temp_min(), k_dist%get_temp_max()

where(p_lay < k_dist%get_press_min()) &
  p_lay = k_dist%get_press_min() + spacing(k_dist%get_press_min())

if(.not. k_dist%source_is_internal()) &
  stop "rrtmgp_rfmip_lw: k-distribution file isn't LW"

nbnd = k_dist%get_nband()
ngpt = k_dist%get_ngpt()

Appending to rrtmgp_rfmip_lw.F90


RRTMGP won't run with pressure less than its minimum. The top level in the RFMIP file is set to 10<sup>-3</sup> Pa. Here we pretend the layer is just a bit less deep. This introduces an error but shows input sanitizing.

In [66]:
%%writefile -a {FILE}

if (top_at_1) then
  p_lev(1,:,:) = &
    k_dist%get_press_min() + epsilon(k_dist%get_press_min())
else
  p_lev(nlay+1,:,:) = &
    k_dist%get_press_min() + epsilon(k_dist%get_press_min())
end if

!# print *, " shape play", shape(p_lay)
!# print *, "play sfc", maxval(p_lay(nlay,:,:)), "tlay sfc", &
!#          maxval(t_lay(nlay,:,:))

Appending to rrtmgp_rfmip_lw.F90


Allocate space for output fluxes (accessed via pointers in `ty_fluxes_broadband`), gas optical properties, and source functions. The `%alloc()` routines carry along the spectral discretization from the k-distribution.

In [67]:
%%writefile -a {FILE}

allocate(flux_up(nlay+1, block_size, nblocks), &
         flux_dn(nlay+1, block_size, nblocks))

!# Allocate g-point fluxes if desired
if (do_gpt_flux) then
  allocate(gpt_flux_up(ngpt, nlay+1, block_size, nblocks), &
           gpt_flux_dn(ngpt, nlay+1, block_size, nblocks))
end if

allocate(sfc_emis_spec(nbnd, block_size))

call stop_on_err(source%alloc(block_size, nlay, k_dist))
call stop_on_err(optical_props%alloc_1scl(block_size, nlay, k_dist))

Appending to rrtmgp_rfmip_lw.F90


In [ ]:
%%writefile -a {FILE}

if (use_rrtmgp_nn) then
  print *, "Starting clear-sky LW computations, using neural &
           &networks as RRTMGP kernel"
else
  print *, "Starting clear-sky LW computations, using &
           &lookup-table as RRTMGP kernel"
end if

In [ ]:
%%writefile -a {FILE}

call system_clock(count_rate=count_rate)
call system_clock(iTime1)

<hr style="height:10px;border-width:0;background-color:blue">

### Loop over blocks

In [ ]:
%%writefile -a {FILE}

do b = 1, nblocks

In [ ]:
%%writefile -a {FILE}

fluxes%flux_up => flux_up(:,:,b)
fluxes%flux_dn => flux_dn(:,:,b)    
if (do_gpt_flux) then
  fluxes%gpt_flux_up => gpt_flux_up(:,:,:,b)
  fluxes%gpt_flux_dn => gpt_flux_dn(:,:,:,b)
end if

Appending to rrtmgp_rfmip_lw.F90


Expand the spectrally-constant surface emissivity to a per-band emissivity for each column 

In [70]:
%%writefile -a {FILE}

do icol = 1, block_size
  do ibnd = 1, nbnd
    sfc_emis_spec(ibnd,icol) = sfc_emis(icol,b)
  end do
end do

Appending to rrtmgp_rfmip_lw.F90


Compute the optical properties of the atmosphere and the Planck source functions from pressures, temperatures, and gas concentrations...

In [71]:
%%writefile -a {FILE}

if (use_rrtmgp_nn) then
  call stop_on_err(k_dist%gas_optics(p_lay(:,:,b),        &
                                     p_lev(:,:,b),        &
                                     t_lay(:,:,b),        &
                                     sfc_t(:,b),          &
                                     gas_conc_array(b),   &
                                     optical_props,       &
                                     source,              &
                                     tlev = t_lev(:,:,b), &
                                     neural_nets = neural_nets))
else
  call stop_on_err(k_dist%gas_optics(p_lay(:,:,b),        &
                                     p_lev(:,:,b),        &
                                     t_lay(:,:,b),        &
                                     sfc_t(:,b),          &
                                     gas_conc_array(b),   &
                                     optical_props,       &
                                     source,              &
                                     tlev = t_lev(:,:,b)))
end if

!# print *, "mean of pfrac is:", mean_3d(planck_frac(:,:,:,b))
!# print *, "mean of tau is:", mean_3d(optical_props%tau)
!# print *, "mean of lay_source is:", mean_3d(source%lay_source)
!# print *, "mean of lev_source is:", mean_3d(source%lev_source)

Appending to rrtmgp_rfmip_lw.F90


In [72]:
%%writefile -a {FILE}

call system_clock(iTime2)

Appending to rrtmgp_rfmip_lw.F90


... and compute the spectrally-resolved fluxes, providing reduced values via `ty_fluxes_broadband`

In [73]:
%%writefile -a {FILE}

call stop_on_err(
       rte_lw(
         optical_props, & 
         top_at_1, & 
         source, & 
         sfc_emis_spec, & 
         fluxes, & 
         n_gauss_angles = n_quad_angles, & 
         use_2stream = .false.))

Appending to rrtmgp_rfmip_lw.F90


In [74]:
%%writefile -a {FILE}

end do   !# blocks

Appending to rrtmgp_rfmip_lw.F90


**End of block**

<hr style="height:10px;border-width:0;background-color:gray">

In [75]:
%%writefile -a {FILE}

call system_clock(iTime3)

Appending to rrtmgp_rfmip_lw.F90


In [77]:
%%writefile -a {FILE}

if (nblocks==1) then
  print *, "-----------------------------------------------------------"
  print '(a,f11.4,/,a,f11.4,/,a,f11.4,a)', &
    ' Time elapsed in gas optics:', &
    real(iTime2-iTime1) / real(count_rate), &
    ' Time elapsed in solver:    ', &
    real(iTime3-iTime2) / real(count_rate), &
    ' Time elapsed in total:     ', &
    real(iTime3-iTime1) / real(count_rate)
  print *, "-----------------------------------------------------------"
else
  print *,'Elapsed time on everything ', &
    real(iTime3-iTime1) / real(count_rate)
end if

Appending to rrtmgp_rfmip_lw.F90


In [78]:
%%writefile -a {FILE}

!# Also deallocates arrays on device
call optical_props%finalize()
call source%finalize()

print*,"-----------------------------------------------------------"// &
       "-----------------------------------------------------------"

!# mean of flux_down is: 103.2458
print *, "Mean of flux_down is:", mean_3d(flux_dn)
print *, "Mean of flux_up is:", mean_3d(flux_up)

Appending to rrtmgp_rfmip_lw.F90


Save fluxes?

In [79]:
%%writefile -a {FILE}

if (save_flux) then
  print *, "Attempting to save fluxes to ", flx_file
  call unblock_and_write(trim(flx_file), 'rlu', flux_up)
  call unblock_and_write(trim(flx_file), 'rld', flux_dn)
  print *, "Fluxes saved to ", flx_file
end if 

Appending to rrtmgp_rfmip_lw.F90


Compare fluxes to benchmark line-by-line results, alongside reference RTE+RRTMGP computations

In [ ]:
%%writefile -a {FILE}

if (compare_flux) then

In [ ]:
%%writefile -a {FILE}

print *, "------------------------------------------------------" // &
        "------------------------------------------------------"
print *, "-----COMPARING ERRORS (W.R.T. LINE-BY-LINE) OF NEW " // &
        "RESULTS AND RRTMGP-256  -------"
print *, "------------------------------------------------------" // &
        "------------------------------------------------------"

allocate(rld_ref(nlay+1, ncol, nexp))
allocate(rlu_ref(nlay+1, ncol, nexp))
allocate(rldu_ref(nlay+1, ncol, nexp))
allocate(rld_nn(nlay+1, ncol, nexp))
allocate(rlu_nn(nlay+1, ncol, nexp))
allocate(rldu_nn(nlay+1, ncol, nexp))
allocate(rld_lbl(nlay+1, ncol, nexp))
allocate(rlu_lbl(nlay+1, ncol, nexp))
allocate(rldu_lbl(nlay+1, ncol, nexp))

flx_file_ref = 'output_fluxes/rlud_Efx_RTE-RRTMGP-181204_rad-' // &
               'irf_r1i1p1f1_gn_REF-DP.nc'
flx_file_lbl = 'output_fluxes/rlud_Efx_LBLRTM-12-8_rad-irf_' // &
               'r1i1p1f1_gn.nc'

call unblock(flux_up, rlu_nn)
call unblock(flux_dn, rld_nn)
rldu_nn = rld_nn - rlu_nn

if(nf90_open(trim(flx_file_ref), NF90_NOWRITE, ncid) /= NF90_NOERR) &
  call stop_on_err("read_and_block_gases_ty: can't find file " // &
                   trim(flx_file_ref))

rlu_ref = read_field(ncid, "rlu", nlay+1, ncol, nexp)
rld_ref = read_field(ncid, "rld", nlay+1, ncol, nexp)
rldu_ref = rld_ref - rlu_ref

if(nf90_open(trim(flx_file_lbl), NF90_NOWRITE, ncid) /= NF90_NOERR) &
  call stop_on_err("read_and_block_gases_ty: can't find file " // &
                   trim(flx_file_lbl))

rlu_lbl = read_field(ncid, "rlu", nlay+1, ncol, nexp)
rld_lbl = read_field(ncid, "rld", nlay+1, ncol, nexp)
rldu_lbl = rld_lbl - rlu_lbl

Appending to rrtmgp_rfmip_lw.F90


In [81]:
%%writefile -a {FILE}

print *, "------------- UPWELLING --------------"

print *, "MAE in upwelling fluxes of new result and RRTMGP, " // &
         "present-day:", &
         mae(reshape(rlu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rlu_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
         mae(reshape(rlu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rlu_ref(:,:,1), shape = [1*ncol*(nlay+1)]))

print *, "MAE in upwelling fluxes of new result and RRTMGP, " // &
         "future:", &
         mae(reshape(rlu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
         reshape(rlu_nn(:,:,4), shape = [1*ncol*(nlay+1)])), &
         mae(reshape(rlu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
         reshape(rlu_ref(:,:,4), shape = [1*ncol*(nlay+1)]))

!# bias in upwelling flux of new result and RRTMGP,
!# present-day, top-of-atm.
print *, "Bias in upwelling flux of new result and RRTMGP, " // &
         "present-day, top-of-atm.:", &
         bias(reshape(rlu_lbl(1,:,1), shape = [1*ncol]), &
         reshape(rlu_nn(1,:,1), shape = [1*ncol])), &
         bias(reshape(rlu_lbl(1,:,1), shape = [1*ncol]), &
         reshape(rlu_ref(1,:,1), shape = [1*ncol]))

!# bias in upwelling flux of new result and RRTMGP,
!# future, top-of-atm.
print *, "Bias in upwelling flux of new result and RRTMGP, " // &
         "future, top-of-atm.:", &
         bias(reshape(rlu_lbl(1,:,4), shape = [1*ncol]), &
         reshape(rlu_nn(1,:,4), shape = [1*ncol])), &
         bias(reshape(rlu_lbl(1,:,4), shape = [1*ncol]), &
         reshape(rlu_ref(1,:,4), shape = [1*ncol]))

!# bias in upwelling flux of new result and RRTMGP,
!# ALL EXPS, top-of-atm.
print *, "Bias in upwelling flux of new result and RRTMGP, " // &
         "ALL EXPS, top-of-atm.:", &
         bias(reshape(rlu_lbl(1,:,:), shape = [nexp*ncol]), &
         reshape(rlu_nn(1,:,:), shape = [nexp*ncol])), &
         bias(reshape(rlu_lbl(1,:,:), shape = [nexp*ncol]), &
         reshape(rlu_ref(1,:,:), shape = [nexp*ncol]))

Appending to rrtmgp_rfmip_lw.F90


- Downwelling refers to the downward movement of air or water in the atmosphere or ocean.
- Net flux refers to the net amount of a quantity (such as heat, moisture, or momentum) being transferred between the Earth's surface and the atmosphere over a specific period of time. It is calculated as the difference between the upward flux (upwelling) and the downward flux (downwelling).

In [82]:
%%writefile -a {FILE}

print *, "-------------- DOWNWELLING --------------"

print *, "MAE in downwelling fluxes of new result and RRTMGP, " // &
         "present-day:", &
         mae(reshape(rld_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rld_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
         mae(reshape(rld_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rld_ref(:,:,1), shape = [1*ncol*(nlay+1)]))

!# print *, "MAE in downwelling fluxes of new result and RRTMGP, " // &
!#          "future:", &
!#          mae(reshape(rld_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!#          reshape(rld_nn(:,:,4), shape = [1*ncol*(nlay+1)])), &
!#          mae(reshape(rld_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!#          reshape(rld_ref(:,:,4), shape = [1*ncol*(nlay+1)]))

print *, "-------------- NET FLUX --------------"

print *, "Max-vertical-error in net fluxes of new result and RRTMGP, " // &
         "pres.day:", &
         maxval(abs(rldu_lbl(:,:,1) - rldu_nn(:,:,1))), &
         maxval(abs(rldu_lbl(:,:,1) - rldu_ref(:,:,1)))

!# print *, "Max-vertical-error in net fluxes of new result and RRTMGP, " // &
!#          "future:", &
!#          maxval(abs(rldu_lbl(:,:,4) - rldu_nn(:,:,4))), &
!#          maxval(abs(rldu_lbl(:,:,4) - rldu_ref(:,:,4)))

print *, "Max-vertical-error in net fluxes of new result and RRTMGP, " // &
         "future-all:", &
         maxval(abs(rldu_lbl(:,:,17) - rldu_nn(:,:,17))), &
         maxval(abs(rldu_lbl(:,:,17) - rldu_ref(:,:,17)))

Appending to rrtmgp_rfmip_lw.F90


In [ ]:
%%writefile -a {FILE}

!# Print header
print *, "---------"
!# Print MAE for present-day net fluxes
print *, "MAE in net fluxes of new result and RRTMGP, present-day: ", &
         mae(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rldu_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
         mae(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rldu_ref(:,:,1), shape = [1*ncol*(nlay+1)]))
!# Print MAE for future net fluxes
!# print *, "#MAE in net fluxes of new result and RRTMGP, future: ", &
!#        mae(reshape(rldu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_nn(:,:,4), shape = [1*ncol*(nlay+1)])), &
!#        mae(reshape(rldu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_ref(:,:,4), shape = [1*ncol*(nlay+1)]))
!# Print MAE for future-all net fluxes
print *, "MAE in net fluxes of new result and RRTMGP, future-all: ", &
         mae(reshape(rldu_lbl(:,:,17), shape = [1*ncol*(nlay+1)]), &
         reshape(rldu_nn(:,:,17), shape = [1*ncol*(nlay+1)])), &
         mae(reshape(rldu_lbl(:,:,17), shape = [1*ncol*(nlay+1)]), &
         reshape(rldu_ref(:,:,17), shape = [1*ncol*(nlay+1)]))
!# Print MAE for all experiments
print *, "MAE in net fluxes of new result and RRTMGP, ALL EXPS: ", &
         mae(reshape(rldu_lbl(:,:,:), shape = [nexp*ncol*(nlay+1)]), &
         reshape(rldu_nn(:,:,:), shape = [nexp*ncol*(nlay+1)])), &
         mae(reshape(rldu_lbl(:,:,:), shape = [nexp*ncol*(nlay+1)]), &
         reshape(rldu_ref(:,:,:), shape = [nexp*ncol*(nlay+1)]))

In [ ]:
%%writefile -a {FILE}

print *, "---------"
!# Print RMSE for present-day net fluxes
!# print *, "#RMSE in net fluxes of new result and RRTMGP, present-day: ", &
!#        rmse(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
!#        rmse(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_ref(:,:,1), shape = [1*ncol*(nlay+1)]))
!# Print RMSE for present-day net fluxes, surface
!# print *, "RMSE in net fluxes of new result and RRTMGP, present-day, SURFACE: ", &
!#        rmse(reshape(rldu_lbl(nlay+1,:,1), shape = [1*ncol]), &
!#        reshape(rldu_nn(nlay+1,:,1), shape = [1*ncol])), &
!#        rmse(reshape(rldu_lbl(nlay+1,:,1), shape = [1*ncol]), &
!#        reshape(rldu_ref(nlay+1,:,1), shape = [1*ncol]))
!# Print RMSE for present-day net fluxes, TOA
!# print *, "#RMSE in net fluxes of new result and RRTMGP, present-day, TOA: ", &
!#        rmse(reshape(rldu_lbl(1,:,1), shape = [1*ncol]), &
!#        reshape(rldu_nn(1,:,1), shape = [1*ncol])), &
!#        rmse(reshape(rldu_lbl(1,:,1), shape = [1*ncol]), &
!#        reshape(rldu_ref(1,:,1), shape = [1*ncol]))
!# Print RMSE for future-all net fluxes, surface
!# print *, "#RMSE in net fluxes of new result and RRTMGP, future-all, SURFACE: ", &
!#        rmse(reshape(rldu_lbl(nlay+1,:,17), shape = [1*ncol]), &
!#        reshape(rldu_nn(nlay+1,:,17), shape = [1*ncol])), &
!#        rmse(reshape(rldu_lbl(nlay+1,:,17), shape = [1*ncol]), &
!#        reshape(rldu_ref(nlay+1,:,17), shape = [1*ncol]))
!# Print RMSE for pre-industrial net fluxes, surface
!# print *, "#RMSE in net fluxes of new result and RRTMGP, pre-industrial, SURFACE: ", &
!#        rmse(reshape(rldu_lbl(nlay+1,:,2), shape = [1*ncol]), &
!#        reshape(rldu_nn(nlay+1,:,2), shape = [1*ncol])), &
!#        rmse(reshape(rldu_lbl(nlay+1,:,2), shape = [1*ncol]), &
!#        reshape(rldu_ref(nlay+1,:,2), shape = [1*ncol]))

Appending to rrtmgp_rfmip_lw.F90


In [ ]:
%%writefile -a {FILE}

print *, "---------"
!# Bias in net fluxes of new result and RRTMGP, present-day
!# print *, "#bias in net fluxes of new result and RRTMGP, ", &
!#        "present-day: ", &
!#        bias(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
!#        bias(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_ref(:,:,1), shape = [1*ncol*(nlay+1)]))
!# Bias in net fluxes of new result and RRTMGP, SURFACE
!# print *, "#bias in net fluxes of new result and RRTMGP, ", &
!#        "present-day, SURFACE: ", &
!#        bias(reshape(rldu_lbl(nlay+1,:,1), shape = [1*ncol]), &
!#        reshape(rldu_nn(nlay+1,:,1), shape = [1*ncol])), &
!#        bias(reshape(rldu_lbl(nlay+1,:,1), shape = [1*ncol]), &
!#        reshape(rldu_ref(nlay+1,:,1), shape = [1*ncol]))
!# Bias in net fluxes of new result and RRTMGP, future
!# print *, "#bias in net fluxes of new result and RRTMGP, ", &
!#        "future: ", &
!#        bias(reshape(rldu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_nn(:,:,4), shape = [1*ncol*(nlay+1)])), &
!#        bias(reshape(rldu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_ref(:,:,4), shape = [1*ncol*(nlay+1)]))
!# Bias in net fluxes of new result and RRTMGP, future-all
!# print *, "#bias in net fluxes of new result and RRTMGP, ", &
!#        "future-all: ", &
!#        bias(reshape(rldu_lbl(:,:,17), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_nn(:,:,17), shape = [1*ncol*(nlay+1)])), &
!#        bias(reshape(rldu_lbl(:,:,17), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rldu_ref(:,:,17), shape = [1*ncol*(nlay+1)]))
!# Bias in net fluxes of new result and RRTMGP, SURFACE, future-all
print *, "Bias in net fluxes of new result and RRTMGP, ", &
         "future-all, SURFACE: ", &
         bias(reshape(rldu_lbl(nlay+1,:,17), shape = [1*ncol]), &
         reshape(rldu_nn(nlay+1,:,17), shape = [1*ncol])), &
         bias(reshape(rldu_lbl(nlay+1,:,17), shape = [1*ncol]), &
         reshape(rldu_ref(nlay+1,:,17), shape = [1*ncol]))

In [ ]:
%%writefile -a {FILE}

print *, "---------"
!# Radiative forcing error at surface, pre-industrial N2O to present-day
print *, "radiative forcing error at surface, ", &
         "pre-industrial N2O to present-day: ", &
         mean(rld_lbl(nlay+1,:,11) - rld_lbl(nlay+1,:,1)) - &
         mean(rld_nn(nlay+1,:,11) - rld_nn(nlay+1,:,1)), &
         mean(rld_lbl(nlay+1,:,11) - rld_lbl(nlay+1,:,1)) - &
         mean(rld_ref(nlay+1,:,11) - rld_ref(nlay+1,:,1))
!# Radiative forcing error at TOA, pre-industrial N2O to present-day
print *, "radiative forcing error at TOA, ", &
         "pre-industrial N2O to present-day: ", &
         mean(rlu_lbl(1,:,11) - rlu_lbl(1,:,1)) - &
         mean(rlu_nn(1,:,11) - rlu_nn(1,:,1)), &
         mean(rlu_lbl(1,:,11) - rlu_lbl(1,:,1)) - &
         mean(rlu_ref(1,:,11) - rlu_ref(1,:,1))
!# MAE in upwelling fluxes of new result w.r.t RRTMGP, present-day
!# print *, "#MAE in upwelling fluxes of new result w.r.t RRTMGP, ", &
!#        "present-day: ", &
!#        mae(reshape(rlu_ref(:,:,1), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rlu_nn(:,:,1), shape = [1*ncol*(nlay+1)]))
!# MAE in upwelling fluxes of new result w.r.t RRTMGP, present-day, SFC
!# print *, "#MAE in upwelling fluxes of new result w.r.t RRTMGP, ", &
!#        "present-day, SFC: ", &
!#        mae(reshape(rlu_ref(nlay+1,:,1), shape = [1*ncol]), &
!#        reshape(rlu_nn(nlay+1,:,1), shape = [1*ncol]))
!# MAE in downwelling fluxes of new result w.r.t RRTMGP, present-day, SFC
!# print *, "#MAE in downwelling fluxes of new result w.r.t RRTMGP, ", &
!#        "present-day, SFC: ", &
!#        mae(reshape(rld_ref(nlay+1,:,1), shape = [1*ncol]), &
!#        reshape(rld_nn(nlay+1,:,1), shape = [1*ncol]))
!# MAE in downwelling fluxes of new result w.r.t RRTMGP, present-day
!# print *, "#MAE in downwelling fluxes of new result w.r.t RRTMGP, ", &
!#        "present-day: ", &
!#        mae(reshape(rld_ref(:,:,1), shape = [1*ncol*(nlay+1)]), &
!#        reshape(rld_nn(:,:,1), shape = [1*ncol*(nlay+1)]))

Appending to rrtmgp_rfmip_lw.F90


In [ ]:
%%writefile -a {FILE}

print *, "---------"
!# Print MAE for net fluxes with respect to RRTMGP
print *, "MAE in net flux w.r.t RRTMGP ", &
         mae(reshape(rldu_ref(:,:,:), shape = [nexp*ncol*(nlay+1)]), &
         reshape(rldu_nn(:,:,:), shape = [nexp*ncol*(nlay+1)]))
!# Print max difference in downward flux with respect to RRTMGP
print *, "Max-diff in d.w. flux w.r.t RRTMGP ", &
         maxval(abs(rld_ref(:,:,:)-rld_nn(:,:,:)))
!# Print max difference in upward flux with respect to RRTMGP
print *, "Max-diff in u.w. flux w.r.t RRTMGP ", &
         maxval(abs(rlu_ref(:,:,:)-rlu_nn(:,:,:)))
!# Print max difference in net flux with respect to RRTMGP
print *, "Max-diff in net flux w.r.t RRTMGP ", &
         maxval(abs(rldu_ref(:,:,:)-rldu_nn(:,:,:)))

Deallocate arrays:

In [ ]:
%%writefile -a {FILE}

deallocate(rld_ref, rlu_ref, rld_nn, rlu_nn, rld_lbl, rlu_lbl, &
           rldu_ref, rldu_nn, rldu_lbl)

In [ ]:
%%writefile -a {FILE}

end if

Appending to rrtmgp_rfmip_lw.F90


---

In [86]:
%%writefile -a {FILE}

deallocate(flux_up, flux_dn)
print *, "SUCCESS!"

Appending to rrtmgp_rfmip_lw.F90


### Functions declaration part

In [87]:
%%writefile -a {FILE}

contains

!#  Root Mean Square Error.
function rmse(x1,x2) result(res)
  implicit none 
  real(wp), dimension(:), intent(in) :: x1,x2
  real(wp) :: res
  real(wp), dimension(size(x1)) :: diff 
  diff = x1 - x2
  res = sqrt( sum(diff**2)/size(diff) )
end function rmse

!# Mean Absolute Error
function mae(x1,x2) result(res)
  implicit none 
  real(wp), dimension(:), intent(in) :: x1,x2
  real(wp) :: res
  real(wp), dimension(size(x1)) :: diff 
  diff = abs(x1 - x2)
  res = sum(diff, dim=1)/size(diff, dim=1)
end function mae

!# Bias can be measured as the average error.
function bias(x1,x2) result(res)
  implicit none 
  real(wp), dimension(:), intent(in) :: x1,x2
  real(wp) :: mean1,mean2, res 
  mean1 = sum(x1, dim=1)/size(x1, dim=1)
  mean2 = sum(x2, dim=1)/size(x2, dim=1)
  res = mean1 - mean2
end function bias

function mean(x) result(mean1)
  implicit none 
  real(wp), dimension(:), intent(in) :: x
  real(wp) :: mean1
  mean1 = sum(x) / size(x)
end function mean

function mean_2d(x) result(mean2)
  implicit none 
  real(wp), dimension(:,:), intent(in) :: x
  real(wp) :: mean2
  mean2 = sum(x) / size(x)
end function mean_2d

function mean_3d(x) result(mean3)
  implicit none 
  real(wp), dimension(:,:,:), intent(in) :: x
  real(wp) :: mean3
  mean3 = sum(x) / size(x)
end function mean_3d

Appending to rrtmgp_rfmip_lw.F90


### End of program

In [64]:
%%writefile -a {FILE}

end program

Appending to rrtmgp_rfmip_lw.f90


---

## Build the RTE+RRTMGP libraries

Most of the cells below are based on the Notebook `ukk23test01-rfmip-clear-sky.ipynb`.

In [14]:
RLIBDIR ="/home/x/git/radnn/ukk23test01/build/"
%cd {RLIBDIR}

/home/x/git/radnn/ukk23test01/build


In [15]:
%env FC=gfortran
%env FCFLAGS=-ffree-line-length-none -m64 -march=native -O3
%env NCHOME=/usr
%env NFHOME=/usr
%env BLASLIB=openblas

env: FC=gfortran
env: FCFLAGS=-ffree-line-length-none -m64 -march=native -O3
env: NCHOME=/usr
env: NFHOME=/usr
env: BLASLIB=openblas


In [16]:
! make clean

rm -f *.optrpt *.mod *.o librrtmgp.a librte.a libneural.a


In [17]:
! make

gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_rte_kind.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_rte_rrtmgp_config.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_rte_util_array.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/kernels/mo_optical_props_kernels.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_optical_props.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_source_functions.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/kernels/mo_fluxes_broadband_kernels.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_fluxes.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/kernels/mo_rte_solver_kernels.F90
gfortran -ffree-line-length-none -m64 

## Build the examples

In [23]:
%cd /home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky

/home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky


In [91]:
! make clean

VAR="../../"
rm rrtmgp_rfmip_sw rrtmgp_rfmip_lw *.o *.mod *.optrpt
rm: cannot remove 'rrtmgp_rfmip_sw': No such file or directory
rm: cannot remove 'rrtmgp_rfmip_lw': No such file or directory
rm: cannot remove '*.optrpt': No such file or directory
make: [Makefile:142: clean] Error 1 (ignored)


In [93]:
! make

VAR="../../"
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c rrtmgp_rfmip_lw.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -o rrtmgp_rfmip_lw rrtmgp_rfmip_lw.o mo_simple_netcdf.o mo_rfmip_io.o mo_load_coefficients.o ../..//build/librte.a ../..//build/librrtmgp.a ../..//build/libneural.a -Wl,-O2 -Wl,--sort-common -Wl,--as-needed -Wl,-z,relro -Wl,-z,now -Wl,--disable-new-dtags -Wl,--gc-sections -Wl,--allow-shlib-undefined -Wl,-rpath,/home/x/conda/envs/tf2/lib -Wl,-rpath-link,/home/x/conda/envs/tf2/lib -L/home/x/conda/envs/tf2/lib  -L/home/x/conda/envs/tf2/targets/x86_64-linux/lib -L/home/x/conda/envs/tf2/targets/x86_64-linux/lib/stubs -L../..//build -L/usr/lib -L/usr/lib -lrrtmgp -lrte -lneural -lnetcdff -lnetcdf -lopenblas  -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c rrtmgp_rfmip_sw.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -o rrtmgp_rfmip_sw r

## RFMIP-CLEAR-SKY - run manually without NN

In [21]:
%cd /home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky

/home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky


    rrtmgp_rfmip_lw                !# 1
        [block_size]               !# 2
        [rfmip_file]               !# 3
        [k-distribution_file]      !# 4
        [forcing_index (1,2,3)]    !# 5
        [physics_index (1,2)]      !# 6
        [input_output file]"       !# 7
    OR:
    rrtmgp_rfmip_lw
        [block_size]              !# 1
        [rfmip_file]              !# 2
        [k-distribution_file]     !# 3
        [forcing_index]           !# 4
        [physics_index]           !# 5
        [NN_lw_abs_file]          !# 6
        [NN_lw_planck_file]       !# 7

In [94]:
! ./rrtmgp_rfmip_lw \
    8 \
    data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 Usage: 
 rrtmgp_rfmip_lw \
     [block_size] \
     [rfmip_file] \
     [k-distribution_file] \
     [forcing_index (1,2,3)] \
     [physics_index (1,2)]  \
     [input_output file]
 OR:
 rrtmgp_rfmip_lw 
     [block_size] \
     [rfmip_file] \
     [k-distribution_file] \
     [forcing_index] \
     [physics_index] \
     [NN_lw_abs_file] \
     [NN_lw_planck_file]
 input file:data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                
 nexp:          18 ncol:         100 nlay:          60 block_size:           8
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky LW computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything   0.679000020    
 -------------------------------------------------------------------------------------------

In [24]:
PATH = "/home/x/git/radnn/ukk23test01/neural/data/"
LWAB = PATH+"lw-g256-2018-12-04_absorption_58_58.nc"
LWPF = PATH+"lw-g256-2018-12-04_planck_frac_16_16.nc"

In [27]:
! ./rrtmgp_rfmip_lw \
    8 \
    data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1 \
    {LWAB} \
    {LWPF}

 Usage: 
 rrtmgp_rfmip_lw_02 \
     [block_size] \
     [rfmip_file] \
     [k-distribution_file] \
     [forcing_index (1,2,3)] \
     [physics_index (1,2)]  \
     [input_output file]
 OR:
 rrtmgp_rfmip_lw_02 
     [block_size] \
     [rfmip_file] \
     [k-distribution_file] \
     [forcing_index] \
     [physics_index] \
     [NN_lw_abs_file] \
     [NN_lw_planck_file]
 input file:data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                
 nexp:          18 ncol:         100 nlay:          60 block_size:           8
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 loading longwave absorption model from /home/x/git/radnn/ukk23test01/neural/data/lw-g256-2018-12-04_absorption_58_58.nc
 loading Planck fraction model from /home/x/git/radnn/ukk23test01/neural/data/lw-g256-2018-12-04_planck_f

## References

- [UKK23] Ukkonen, P., & Hogan, R. J. (2023). Implementation of a machine-learned gas optics parameterization in the ECMWF Integrated Forecasting System: RRTMGP-NN 2.0. Geoscientific Model Development, 16(11), 3241–3261. https://doi.org/10.5194/gmd-16-3241-2023
- [UKK22] Ukkonen, P. (2022). Exploring Pathways to More Accurate Machine Learning Emulation of Atmospheric Radiative Transfer. Journal of Advances in Modeling Earth Systems, 14(4), e2021MS002875. https://doi.org/10.1029/2021MS002875
- [UKK20] Ukkonen, P., Pincus, R., Hogan, R. J., Pagh Nielsen, K., & Kaas, E. (2020). Accelerating Radiation Computations for Dynamical Models With Targeted Machine Learning and Code Optimization. Journal of Advances in Modeling Earth Systems, 12(12), e2020MS002226. https://doi.org/10.1029/2020MS002226